## Yield From (python 3.3+)
1. This expression actually means "Yield from iteratble". Function with `Yield from` is still a generator. It's  roughly equivalent to
    ```
    yield from Iteratble <==>

    for i in Iterable: 
        yield i
    ```

In [ ]:
# Example 1: yield from range object
def yield_from_gen():
    for i in range(2):
        yield i

for i in yield_from_gen():
    print(i)

# Example 2: yield from a generator
def test(n): 
    i = 0
    while i < n: 
        print("test i: ", i)
        yield i
        i+=1
    print("This will be executed after the yield")

def yield_from_func(n): 
    print("Start: ")
    j = yield from test(n)
    print("j: ", j)
    # 'This will be executed after the yield' will STILL be executed
    # The last value of j will be None, after 
    print("========")
    # You can have another round 
    yield from test(n)
    print("End: ")

for i in yield_from_func(4): 
    print("main loop i: ", i)

2. USE #1 : `Yield from` is to address the woes in `yield`. If you have `return` after a `yield`, the returned value will trigger an implicit `StopIteration`, but its value won't be captured.  
    - `yield from` will capture the returned value. But it will also deliver the explicit `StopIteration`

In [ ]:
# 1
def some_gen(): 
    yield 0
    yield 1
    # You can also do return
    return "Done0"

g = some_gen()
print(next(g))
# now see 1, because generator picks up where it left off.
print(next(g))
try:
    print(next(g))
except StopIteration:
    print("stop iteration is received, no return value")

# 2
print("If directly using for loop, return value won't be returned ")
g = some_gen()
for i in g: 
    print(i)

print("If using yield from, then using for loop, return value WILL be returned")
def test_some_gen():
    g = some_gen()
    res = yield from g
    print(res)
    print("After everything in yield from is ended, what's after will be finished")
for i in test_some_gen():
    print(i)

# 3
tsg = test_some_gen()
print(next(tsg))
print(next(tsg))
try: 
    next(tsg) # this will finish the part after yield from, but will also raise stopiteration
except StopIteration:
    print("If you manually call next() just enough times to finish yielding, you will see the return value, and the rest of the generator")
    



3. USE #2: `yield from` creates a layer to interact with the "sender", as a pipe. Python calls it a "proxy generator"
    - Advance uses of `yield from`: can do `send`, and `close()` (see below).

In [ ]:
def total_average():
    total = 0.0
    count = 0
    avg = None
    print("starting average generator")
    while True:
        num = yield avg
        if num is None:
            break
        total += num
        count += 1
        avg = total/count

def wrap_average(average_generator):
    """This is just a pipe to the generator"""
    print("starting wrap_average")
    avg = yield from average_generator

# Note: total_average() is the generator object. total_average is generator function
w = wrap_average(total_average())
# Execute everthing until hitting the first iterator. None being returned
print("starting the generator: ", next(w))
print(w.send(3))
print(w.send(4))
# Finish Coroutine
# ? Not sure why w.send(None) is giving me StopIteration? - yield from can still deliver StopIteration
# w.send(None)
try: 
    if w.send(5) == 4:
        w.throw(ValueError("I don't like this value"))
except ValueError:
    pass
w.close()
        
    

Use #3: handling exceptions: 

In [ ]:
class SpamException(Exception):
    pass

def writer():
    while True:
        try:
            w = (yield)
        except SpamException:
            print('***')
        else:
            print('>> ', w)


def writer_wrapper(coro):
    """Works. Manually catches exceptions and throws them"""
    coro.send(None)  # prime the coro
    while True:
        try:
            try:
                x = (yield)
            except Exception as e:   # This catches the SpamException
                print("wrapper caught exception: ", e)
                coro.throw(e)
            else:
                print("wrapper: x", x)
                coro.send(x)
        except StopIteration:
            pass


w_generator = writer_wrapper(writer())
next(w_generator)
# Note: if you send the SpamException, it won't be in the exception handling system.
w_generator.send(SpamException("haha"))
w_generator.throw(SpamException("haha"))

But with `yield from` this could be simplified to

In [ ]:
def writer_wrapper(coro):
    yield from coro

w_generator = writer_wrapper(writer())
next(w_generator)
w_generator.throw(SpamException("Haha"))

In [ ]:

import functools
import types
def fake_coroutine(func):
    # make use of the nice features of yield from
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        rs = func(*args, **kwargs)
        # res is any return value at the very end of rs
        res = yield from rs
        return res
    wrapper = types.coroutine(wrapper)
    return wrapper

@fake_coroutine
def multiplication_server():
    res = None
    while True:
        a, b = yield res
        res = a * b

def multiplication_client(c):
    # start the server by executing up to surpass first yield call
    next(c)
    # Send values to yield as a, b, do multiplication, then yield the result back
    print(c.send((1,2)))
    print(c.send((3,4)))
    c.close()

c = multiplication_server()
multiplication_client(c)


1. asyncio.coroutine is a wrapper that calls next() on the function. Python3.4 - 3.10
    - So it's basically a middle man, emit all values from yield, and catch return value
    - Of course, if you call it manually, return value will surface after StopIteration
    - this is where "async.run_until_complete" becomes handy


In [ ]:
# Ipython / Jupyter Notebook does not play well with coroutine. Run this in a real python kernel
import asyncio
@asyncio.coroutine
def multiplication_server():
    res = None
    while True:
        a, b = yield res
        res = a * b

@asyncio.coroutine
def multiplication_client(c):
    # start the server by executing up to surpass first yield call
    next(c)
    # Send values to yield as a, b, do multiplication, then yield the result back
    print(c.send((1,2)))
    print(c.send((3,4)))
    c.close()

c = multiplication_server()
loop = asyncio.get_event_loop()
loop.run_until_complete(multiplication_client(c))
loop.close()

`async def` and `await`. `async def` is equivalent to `asyncio.coroutine`, and `await` is equivalent to `yield from`